In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport
from autogluon.tabular import TabularPredictor,TabularDataset
import os

d:\worker_code\.venvgis\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
d:\worker_code\.venvgis\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [3]:
model_save_path = 'D:\ArcgisData\pred_soildeep\pred_model'

In [ ]:
# 读取数据
file_path = r"F:\cache_data\pre_property_table\dy\feature_deep_dy.csv"
data = pd.read_csv(file_path)
data


In [5]:
data.drop_duplicates(inplace=True)
data.describe()

,纬度,经度,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,...,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,MRRTF,MRVBF
count,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,4045.000000,...,4045.000000,4045.000000,4045.000000,4045.000000,4.045000e+03,4045.000000,4045.000000,4045.000000,4.045000e+03,4.045000e+03
mean,26.140514,107.478658,983.937650,0.877816,3.122658,963.747094,20.766982,0.576427,0.609489,15.806813,...,1.500321,1.717450,192.473887,447863.248455,2.892516e+06,5.612929,5.304079,11.197775,3.116610e-01,5.277244e-01
std,0.120368,0.159207,183.136486,0.313103,1.705540,177.208801,24.616782,4.177373,6.461031,16.769686,...,0.061207,0.091255,1.540876,15931.121497,1.332407e+04,0.407243,3.473368,11.400364,6.122378e-01,7.778537e-01
min,25.857301,107.129531,572.100000,0.033098,0.000000,573.533700,-0.150024,0.000000,-51.953915,0.000000,...,1.280000,1.541667,187.725100,412900.000000,2.861150e+06,4.447507,2.000000,1.000000,1.132427e-14,7.038814e-14
25%,26.054113,107.366445,852.600000,0.678410,1.741530,839.952700,4.404907,0.000000,-0.840810,7.154013,...,1.469167,1.658333,191.639940,436630.000000,2.882925e+06,5.337762,3.000000,7.000000,1.111463e-02,2.934986e-02
50%,26.127315,107.467543,943.600000,0.867519,3.124044,924.068400,13.046570,0.000000,0.658295,12.993794,...,1.512500,1.691667,192.267040,446735.000000,2.891085e+06,5.543946,4.000000,7.000000,1.074371e-01,2.668618e-01
75%,26.231915,107.612926,1082.300000,1.094032,4.497582,1058.234400,27.835205,0.000000,2.326719,20.633993,...,1.544167,1.766667,193.283170,461270.000000,2.902610e+06,5.822039,6.000000,9.000000,2.995126e-01,5.775232e-01
max,26.414724,107.772751,1660.200000,2.131159,6.283186,1628.765500,289.642640,60.401306,77.550780,455.575260,...,1.623333,2.066667,197.809980,477310.000000,2.922890e+06,7.361616,14.000000,42.000000,4.961693e+00,5.093605e+00


In [6]:
data.columns

Index(['HD', '纬度', '经度', '岩石类型', 'DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT', 'PH', 'DZ', 'DL',
       'MRRTF', 'MRVBF'],
      dtype='object')

In [8]:
data = data[['LON','LAT','HD','DEM','Aspect','PlanCurvature','ProfileCurvature','Slope','TopographicWetnessIndex','ValleyDepth','ndvi','PCA_0','MRVBF','DL']]
data['HD'] = data['HD'].astype('category')
data['DL'] = data['DL'].astype('category')
data.dtypes

LON                         float64
LAT                         float64
HD                         category
DEM                         float64
Aspect                      float64
PlanCurvature               float64
ProfileCurvature            float64
Slope                       float64
TopographicWetnessIndex     float64
ValleyDepth                 float64
ndvi                        float64
PCA_0                       float64
MRVBF                       float64
DL                         category
dtype: object

In [9]:
train_data, test_data = train_test_split(data,test_size=0.2,random_state=42)

In [10]:
label = 'HD'

In [12]:
predictor = TabularPredictor(label=label,eval_metric='f1_weighted',path=os.path.join(model_save_path,"test0612"),problem_type='multiclass')
predictor.fit(train_data,tuning_data=test_data,time_limit=10800)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "D:\ArcgisData\pred_soildeep\pred_model\test0612"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       44.59 GB / 63.81 GB (69.9%)
Disk Space Avail: 

In [13]:
predictor.leaderboard(train_data,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.994442,0.643705,f1_weighted,0.084786,0.016971,2.749868,0.084786,0.016971,2.749868,1,True,5
1,ExtraTreesEntr,0.994442,0.588774,f1_weighted,0.268366,0.035001,0.497978,0.268366,0.035001,0.497978,1,True,10
2,RandomForestEntr,0.994441,0.623923,f1_weighted,0.183340,0.034868,0.597418,0.183340,0.034868,0.597418,1,True,7
3,RandomForestGini,0.994441,0.621857,f1_weighted,0.194994,0.033410,0.566340,0.194994,0.033410,0.566340,1,True,6
4,ExtraTreesGini,0.994441,0.589747,f1_weighted,0.232188,0.034299,0.498991,0.232188,0.034299,0.498991,1,True,9
5,LightGBMLarge,0.994441,0.647712,f1_weighted,0.033905,0.007005,3.140921,0.033905,0.007005,3.140921,1,True,13
6,WeightedEnsemble_L2,0.993824,0.690897,f1_weighted,0.931744,0.120581,18.975028,0.000000,0.001172,0.235604,2,True,14
7,KNeighborsDist,0.993517,0.681359,f1_weighted,0.000667,0.000000,0.004945,0.000667,0.000000,0.004945,1,True,2
8,LightGBMXT,0.889324,0.628553,f1_weighted,0.041365,0.013763,2.469623,0.041365,0.013763,2.469623,1,True,4
9,CatBoost,0.848533,0.635277,f1_weighted,0.001089,0.000000,22.899670,0.001089,0.000000,22.899670,1,True,8


In [14]:
predictor.leaderboard(test_data,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.686921,0.690897,f1_weighted,0.307956,0.120581,18.975028,0.015627,0.001172,0.235604,2,True,14
1,KNeighborsDist,0.677592,0.681359,f1_weighted,0.002000,0.000000,0.004945,0.002000,0.000000,0.004945,1,True,2
2,KNeighborsUnif,0.671786,0.675511,f1_weighted,0.002999,0.004115,3.183639,0.002999,0.004115,3.183639,1,True,1
3,LightGBMLarge,0.643984,0.647712,f1_weighted,0.016642,0.007005,3.140921,0.016642,0.007005,3.140921,1,True,13
4,LightGBM,0.639975,0.643705,f1_weighted,0.053690,0.016971,2.749868,0.053690,0.016971,2.749868,1,True,5
5,XGBoost,0.639064,0.642782,f1_weighted,0.033335,0.000000,1.566693,0.033335,0.000000,1.566693,1,True,11
6,CatBoost,0.631638,0.635277,f1_weighted,0.002068,0.000000,22.899670,0.002068,0.000000,22.899670,1,True,8
7,LightGBMXT,0.624904,0.628553,f1_weighted,0.034285,0.013763,2.469623,0.034285,0.013763,2.469623,1,True,4
8,RandomForestEntr,0.620330,0.623923,f1_weighted,0.083746,0.034868,0.597418,0.083746,0.034868,0.597418,1,True,7
9,RandomForestGini,0.618264,0.621857,f1_weighted,0.109587,0.033410,0.566340,0.109587,0.033410,0.566340,1,True,6
